In [2]:
import matplotlib.pyplot as plt

from pyrocko import util, model, io, trace, moment_tensor, gmtpy
from pyrocko import pz
from pyrocko import orthodrome as od
from pyrocko.io import quakeml
from pyrocko.io import stationxml as fdsn
from pyrocko.client import catalog
from pyrocko.automap import Map

from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
from obspy.core.event import Catalog
from obspy.core.stream import Stream
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude
from obspy import read
from obspy import read_events
from obspy import read_inventory
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import pickle

import geopy.distance


workdir='../'
catdir =  os.path.join(workdir,'CAT')
meta_datadir=os.path.join(workdir,'META_DATA')
datadir=os.path.join(workdir,'DATA')

In [ ]:
#turkey eq parameters
eq_name='turkey_'
stime=UTCDateTime('2023-02-01T00:00:00')
etime=UTCDateTime('2023-02-10T00:00:00')

min_lat=30.0
max_lat=45.0
min_long=25.0
max_long=45.0

In [ ]:
#cayman eq parameters
eq_name='cayman_'
stime=UTCDateTime('2020-01-27T00:00:00')
etime=UTCDateTime('2020-01-29T00:00:00')

min_lat=17.0
max_lat=21.0
min_long=-80.0
max_long=-76.0

In [ ]:
#DOWNLOAD CATALOGUE
client=Client('IRIS')

cat_eq=client.get_events(starttime=stime,endtime=etime,
                minlatitude=min_lat, maxlatitude=max_lat,
                minlongitude=min_long, maxlongitude=max_long,
                minmagnitude=7.0)

print(cat_eq)

In [ ]:
# reformat local catalog from INGV
catname = os.path.join(catdir, 'cat_eq_' + eq_name[:-1] + '.pf')

events = []

for ev in cat_eq:
    timetmp= str(ev.origins[0].time)
    timestr = timetmp[0:10] + ' ' + timetmp[11:23]                                        # format: '2010-01-01 00:00:00.000'
    time = util.str_to_time(timestr)
    event_name = eq_name + timestr[0:4] +'_'+ timestr[5:7]+'_'+timestr[8:10]+'_'+timestr[11:13]+'_'+timestr[14:16]+'_'+timestr[17:19]

    lat=    float( ev.origins[0].latitude )
    lon =   float( ev.origins[0].longitude )
    depth = float( ev.origins[0].depth )
    magnitude = float( ev.magnitudes[0].mag )
    events.append(model.Event(name=event_name, time=time,
                              lat=lat, lon=lon,
                              depth=depth, magnitude=magnitude))
events.sort(key=lambda x: x.time, reverse=False)
model.dump_events(events, catname)
print('Number of events:', len(events))

In [ ]:
#read catalogue
catname = os.path.join(catdir, 'cat_eq_' + eq_name[:-1] + '.pf')

cat = model.load_events(catname)
print('Number of events:',len(cat))
print('Name of events:',cat[0].name)


In [ ]:
client=Client('INGV')
stations_name=os.path.join(meta_datadir, 'stations_flegrei_INGV.xml')
stations=read_inventory(stations_name)                                 #read

print(stations)

In [ ]:
#WAVEFORMS
datadir_2=os.path.join(workdir,'DATA_big_eq')

for ev in cat:
    evID=ev.name

    #transform UTC time
    t = util.time_to_str(ev.time)

    print('origin UTC time event:',t)
    print('extimated magnitude:',ev.magnitude)

    event_start = UTCDateTime(t) 
    #print('event starts at:',event_start)

    event_end=UTCDateTime(t) +9000
    #print('event ends at:',event_end)


    wave=Stream()
    for network in stations:
        for  station in network.stations:
            try:
                wave += client.get_waveforms(starttime=event_start,endtime=event_end,
                                    network=network.code,station=station.code,location='*', channel='HH?',
                                    attach_response=True)
            except:
                #print(station.code , 'station not recording')
                continue

    print('traces found:',len(wave.traces))

    #remove instrumental response
    pre_filt = [0.005, 0.01, 25,30]
    
    #remove instrumental response
    wave.remove_response(inventory=stations, output='DISP', pre_filt=pre_filt)

    waveletdir=os.path.join(datadir_2,evID)
    
    os.mkdir(waveletdir)

    wavelet_name= os.path.join(waveletdir,evID)  
    wave.write(wavelet_name +'.mseed',format='MSEED')
    print('saved!')

# FLEGREI TEST

In [ ]:
catname = os.path.join(catdir, 'catologue_flegrei_new_mag2_5.pf')

cat = model.load_events(catname)
print('Number of events:', len(cat))

In [ ]:
datadir_3=os.path.join(workdir,'DATA_response')

for ev in cat:
    evID=ev.name

    #transform UTC time
    t = util.time_to_str(ev.time)

    print('origin UTC time event:',t)
    print('extimated magnitude:',ev.magnitude)

    event_start = UTCDateTime(t) - 20
    #print('event starts at:',event_start)

    event_end=UTCDateTime(t) +40
    #print('event ends at:',event_end)


    wave=Stream()
    for network in stations:
        for  station in network.stations:
            try:
                wave += client.get_waveforms(starttime=event_start,endtime=event_end,
                                    network=network.code,station=station.code,location='*', channel='HH?',
                                    attach_response=True)
            except:
                #print(station.code , 'station not recording')
                continue

    print('traces found:',len(wave.traces))

    #remove instrumental response
    pre_filt = [0.01, 0.1, 25,30]
    
    #remove instrumental response
    wave.remove_response(inventory=stations, output='DISP', pre_filt=pre_filt)

    waveletdir=os.path.join(datadir_3,evID)
    
    os.mkdir(waveletdir)

    wavelet_name= os.path.join(waveletdir,evID)  
    wave.write(wavelet_name +'.mseed',format='MSEED')
    print('saved!')